In [89]:
pip install --upgrade ipywidgets


Note: you may need to restart the kernel to use updated packages.


# Data Visualization using the interactive dashboard usin the ipywidgets.

## Customize Matplotlib plots with interactive widgets (e.g., sliders, buttons) using libraries like ipywidgets.

In [17]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [19]:
# Load dataset
df = sns.load_dataset('penguins')

In [21]:
# Display first 5 rows
print("First 5 rows:")
display(df.head())


First 5 rows:


species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
3          NaN     NaN  
4       3450.0  Female

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [25]:
df.describe()

bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
count      342.000000     342.000000         342.000000   342.000000
mean        43.921930      17.151170         200.915205  4201.754386
std          5.459584       1.974793          14.061714   801.954536
min         32.100000      13.100000         172.000000  2700.000000
25%         39.225000      15.600000         190.000000  3550.000000
50%         44.450000      17.300000         197.000000  4050.000000
75%         48.500000      18.700000         213.000000  4750.000000
max         59.600000      21.500000         231.000000  6300.000000

In [27]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [29]:
# Loop through each column and display unique values for categorical columns only
for col in df.columns:
    if df[col].dtype == 'object':   # Check column data type
        unique_vals = df[col].unique()          # Get unique values
        num_unique = df[col].nunique()          # Count of unique values
        print(f"Column: {col}")
        print(f"Number of unique values: {num_unique}")
        print(f"Unique values: {unique_vals}\n")


Column: species
Number of unique values: 3
Unique values: ['Adelie' 'Chinstrap' 'Gentoo']

Column: island
Number of unique values: 3
Unique values: ['Torgersen' 'Biscoe' 'Dream']

Column: sex
Number of unique values: 2
Unique values: ['Male' 'Female' nan]



In [31]:
# Fill numerical columns with median
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_5460\3524589151.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)


In [33]:
# Fill categorical columns with mode
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    mode_val = df[col].mode()[0]  # mode() returns a Series, take first element
    df[col].fillna(mode_val, inplace=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_5460\1941861034.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)


In [76]:
# Check missing values after filling
print("\nMissing values after filling:\n")
print(df.isnull().sum())


Missing values after filling:

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64


In [78]:
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, Checkbox, Button, HBox, VBox, Layout, Output, interactive_output
from IPython.display import display, clear_output

plt.style.use('seaborn-v0_8')
num_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
cat_features = ['species', 'island', 'sex']


In [80]:
# Scatter controls
scatter_x = Dropdown(options=num_features, value='bill_length_mm', description='X-axis:', layout=Layout(width='22%'))
scatter_y = Dropdown(options=num_features, value='bill_depth_mm', description='Y-axis:', layout=Layout(width='22%'))
scatter_color = Checkbox(value=True, description='Color by Species', layout=Layout(width='12%'))
scatter_filter = FloatRangeSlider(value=[df['bill_length_mm'].min(), df['bill_length_mm'].max()],
                                  min=df['bill_length_mm'].min(), max=df['bill_length_mm'].max(),
                                  step=0.5, description='Bill Length:', layout=Layout(width='40%'))

# Histogram controls
hist_feature = Dropdown(options=num_features, value='flipper_length_mm', description='Feature:', layout=Layout(width='30%'))
hist_bins = IntSlider(value=20, min=5, max=50, step=1, description='Bins:', layout=Layout(width='40%'))
hist_norm = Checkbox(value=False, description='Normalize', layout=Layout(width='20%'))

# Bar chart controls
bar_metric = Dropdown(options=num_features, value='body_mass_g', description='Metric:', layout=Layout(width='30%'))
bar_group = Dropdown(options=['species','island'], value='species', description='Group by:', layout=Layout(width='30%'))

# Reset button
reset_btn = Button(description='Reset All', button_style='info', layout=Layout(width='150px'))

# Summary box
summary_box = Output(layout=Layout(border='1px solid gray', padding='10px', width='100%'))


In [82]:
# Filter data based on slider
def filter_data():
    filtered = df[(df['bill_length_mm'] >= scatter_filter.value[0]) & 
                  (df['bill_length_mm'] <= scatter_filter.value[1])]
    return filtered

# Update summary statistics
def update_summary(data):
    with summary_box:
        summary_box.clear_output()
        display("Summary Statistics (Filtered Data):")
        display(data[num_features].describe().T)
        display("\nGroup-wise Averages:")
        display(data.groupby('species')[num_features].mean().round(2))


In [84]:
def update_plots(**kwargs):
    clear_output(wait=True)
    data = filter_data()
    update_summary(data)
    
    # ----------------- Scatter Plot -----------------
    fig1, ax1 = plt.subplots(figsize=(16,9))
    x_feat = scatter_x.value
    y_feat = scatter_y.value
    if scatter_color.value:
        for species, group in data.groupby('species'):
            ax1.scatter(group[x_feat], group[y_feat], label=species, s=60, alpha=0.8)
            max_idx = group[y_feat].idxmax()
            ax1.annotate(f"{group.loc[max_idx,'species']}", 
                         (group.loc[max_idx, x_feat], group.loc[max_idx, y_feat]),
                         textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)
        ax1.legend(title='Species')
    else:
        ax1.scatter(data[x_feat], data[y_feat], color='steelblue', s=60, alpha=0.8)
    ax1.set_xlabel(x_feat)
    ax1.set_ylabel(y_feat)
    ax1.set_title(f'Scatter: {x_feat} vs {y_feat}', fontsize=16)
    ax1.grid(True)
    
    # Highlight filtered points in histogram
    fig2, ax2 = plt.subplots(figsize=(16,9))
    feature = hist_feature.value
    bins = hist_bins.value
    density = hist_norm.value
    counts, bins_edges, patches = ax2.hist(df[feature], bins=bins, density=density, color='lightgray', alpha=0.5, edgecolor='black')
    ax2.hist(data[feature], bins=bins, density=density, color='skyblue', edgecolor='black')
    mean_val = data[feature].mean()
    ax2.axvline(mean_val, color='red', linestyle='dashed', label=f'Mean: {mean_val:.2f}')
    ax2.set_xlabel(feature)
    ax2.set_ylabel('Density' if density else 'Count')
    ax2.set_title(f'Histogram of {feature}', fontsize=16)
    ax2.legend()
    
    # ----------------- Bar Chart -----------------
    fig3, ax3 = plt.subplots(figsize=(16,9))
    metric = bar_metric.value
    group_col = bar_group.value
    grouped = data.groupby(group_col)[metric].agg(['mean','std']).reset_index()
    colors = plt.cm.Set2(np.linspace(0,1,len(grouped)))
    bars = ax3.bar(grouped[group_col], grouped['mean'], yerr=grouped['std'], color=colors, capsize=5)
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2.0, height, f'{height:.1f}', ha='center', va='bottom', fontsize=10)
    ax3.set_xlabel(group_col.capitalize())
    ax3.set_ylabel(f'Average {metric}')
    ax3.set_title(f'{metric} by {group_col}', fontsize=16)
    plt.xticks(rotation=45)
    ax3.grid(axis='y')
    
    # ----------------- Additional Insights -----------------
    # Boxplot
    fig4, ax4 = plt.subplots(figsize=(16,4))
    sns.boxplot(x='species', y=num_features[0], data=data, palette='Set2', ax=ax4)
    ax4.set_title(f'Boxplot of {num_features[0]} by Species', fontsize=14)
    
    # Correlation Heatmap
    fig5, ax5 = plt.subplots(figsize=(10,6))
    sns.heatmap(data[num_features].corr(), annot=True, cmap='coolwarm', ax=ax5)
    ax5.set_title('Correlation Heatmap of Numerical Features', fontsize=14)

    plt.close(fig1)
    plt.close(fig2)
    plt.close(fig3)
    plt.close(fig4)
    plt.close(fig5)
    display(summary_box, fig1, fig2, fig3, fig4, fig5)


In [86]:
def reset_all(button):
    scatter_x.value = 'bill_length_mm'
    scatter_y.value = 'bill_depth_mm'
    scatter_color.value = True
    scatter_filter.value = [df['bill_length_mm'].min(), df['bill_length_mm'].max()]
    hist_feature.value = 'flipper_length_mm'
    hist_bins.value = 20
    hist_norm.value = False
    bar_metric.value = 'body_mass_g'
    bar_group.value = 'species'
    print("Filters reset!")
    update_plots()

reset_btn.on_click(reset_all)


In [88]:
scatter_controls = HBox([scatter_x, scatter_y, scatter_color, scatter_filter])
hist_controls = HBox([hist_feature, hist_bins, hist_norm])
bar_controls = HBox([bar_metric, bar_group])
controls = VBox([scatter_controls, hist_controls, bar_controls, reset_btn])

widget_dict = {
    'scatter_x': scatter_x,
    'scatter_y': scatter_y,
    'scatter_color': scatter_color,
    'scatter_filter': scatter_filter,
    'hist_feature': hist_feature,
    'hist_bins': hist_bins,
    'hist_norm': hist_norm,
    'bar_metric': bar_metric,
    'bar_group': bar_group
}

out = interactive_output(update_plots, widget_dict)
display(controls, out)


Output()

# 🐧 Penguin Interactive Dashboard — User Guide & Insights

This dashboard is built using the **Palmer Penguins dataset**, which contains measurements of three penguin species: **Adelie, Chinstrap, and Gentoo**.  
It allows us to explore relationships between physical traits, distributions of measurements, and group-level comparisons.  

The dashboard is interactive — you can **change dropdowns, sliders, and checkboxes** to explore different views of the data in real time.

---

## 🔑 How to Use the Dashboard

### 1. **Scatter Plot (Relationship Explorer)**
- **X-axis / Y-axis dropdowns:** Select any two numerical features (e.g., bill length, flipper length, body mass).
- **Color by Species checkbox:** Turn on to color-code by penguin species.
- **Bill Length filter slider:** Restrict analysis to penguins within a chosen bill length range.
- **What to Look For:**
  - Whether species form **separate clusters** (helps in species classification).
  - Whether traits are **positively or negatively correlated**.
  - Outliers — penguins with unusual measurements.

---

### 2. **Histogram (Distribution Analyzer)**
- **Feature dropdown:** Choose one trait (e.g., flipper length, body mass).
- **Bins slider:** Adjust bin size for more or less granularity.
- **Normalize checkbox:** Switch between raw counts vs. probability density.
- **Red dashed line:** Shows the **mean value** of the feature.
- **What to Look For:**
  - Whether the distribution is **normal (bell-shaped)** or skewed.
  - How much **variation** exists in each measurement.
  - Comparison of penguin sizes (body mass, flipper length, etc.).

---

### 3. **Bar Chart (Grouped Averages)**
- **Metric dropdown:** Select a numerical feature (e.g., body mass, flipper length).
- **Group by dropdown:** Choose `species` or `island`.
- **Error bars:** Represent **standard deviation**, showing how spread out the values are.
- **What to Look For:**
  - Which species are **larger/heavier** on average.
  - Differences between penguins from different **islands**.
  - Which trait shows the **most variability** between groups.

---

### 4. **Summary Statistics Panel**
- Automatically updates to show:
  - Overall **mean, median, standard deviation** for the filtered data.
  - **Species-wise averages** for quick comparison.
- **What to Look For:**
  - How filtering (e.g., narrowing bill length range) changes group statistics.
  - Quick reference for comparing multiple features.

---

### 5. **Reset Button**
- Resets all widgets to their default values.
- Reloads the full dataset view.
- Useful when you get lost in filtering and want to start fresh.

---

## 📊 Key Insights from the Penguins Dataset

1. **Species differences:**
   - **Gentoo penguins** are generally the **largest and heaviest**.
   - **Adelie penguins** are smaller, with shorter flippers and lower body mass.
   - **Chinstrap penguins** often overlap with Adelie in size, but have slightly longer bills.

2. **Correlations:**
   - **Flipper length and body mass** are positively correlated: larger penguins tend to weigh more.
   - **Bill length vs. depth** helps distinguish species (Adelie tend to have shorter, deeper bills).

3. **Distribution shapes:**
   - Many features (e.g., body mass) are roughly **bell-shaped**, but some are skewed.
   - Variation differs by species: Gentoo body mass varies more than Adelie’s.

4. **Geographical variation:**
   - Penguins from different **islands** may show differences in body size and traits.
   - Island habitat could influence diet and growth.

5. **Practical takeaway:**
   - These measurements can be used to **classify penguin species**.
   - The dashboard makes it easy to see which features separate species best.

---

## 💡 Tips for Exploration
- Try setting **X = flipper_length_mm** and **Y = body_mass_g** → clear species clusters appear.
- Use the **bill length filter** to see how removing outliers changes averages.
- Compare **species vs. island** in the bar chart → shows if environment influences traits.
- Normalize histograms to compare **relative distributions** between features.

---

📌 **In summary:**  
This dashboard is a tool to visually and interactively explore how penguin traits vary across species and islands.  
It helps us understand **relationships, distributions, and group differences** in an intuitive way, bridging both technical and non-technical insights.


In [92]:
# Generate a Word report for the Penguin Dashboard (no kaleido used)
# Save this as a single cell in your Jupyter Notebook and run it.
# Requirements:
#   pip install seaborn pandas matplotlib numpy python-docx

from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

# -------------------------
# SETTINGS
# -------------------------
OUT_DIR = Path('./penguin_report')
OUT_DIR.mkdir(parents=True, exist_ok=True)
DOC_PATH = OUT_DIR / 'Penguin_Dashboard_Report.docx'

# -------------------------
# LOAD DATA (with fallback)
# -------------------------
try:
    # Primary: load via seaborn (internet required)
    df = sns.load_dataset('penguins')
    if df is None:
        raise Exception("seaborn returned None")
except Exception as e:
    print("Warning: seaborn.load_dataset failed (maybe no internet).")
    print("Attempting to load from local 'penguins.csv' (if available).")
    try:
        df = pd.read_csv('penguins.csv')
    except Exception as e2:
        raise RuntimeError(
            "Unable to load dataset. Please either enable internet or place 'penguins.csv' in the current folder."
        )

# Drop missing values for simplicity (same as dashboard)
df.dropna(inplace=True)

num_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# -------------------------
# GENERATE REPRESENTATIVE FIGURES (Matplotlib/Seaborn)
# -------------------------
def save_figures(output_dir):
    # Scatter
    fig1_path = output_dir / 'scatter.png'
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=df, x='flipper_length_mm', y='body_mass_g', hue='species', s=80)
    plt.title('Scatter: Flipper Length vs Body Mass by Species')
    plt.tight_layout()
    plt.savefig(fig1_path, dpi=150)
    plt.close()

    # Histogram
    fig2_path = output_dir / 'histogram.png'
    plt.figure(figsize=(10,6))
    plt.hist(df['body_mass_g'], bins=20, edgecolor='black', color='skyblue')
    plt.axvline(df['body_mass_g'].mean(), color='red', linestyle='dashed',
                label=f"Mean = {df['body_mass_g'].mean():.1f}")
    plt.title('Histogram: Body Mass (g)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(fig2_path, dpi=150)
    plt.close()

    # Bar chart
    fig3_path = output_dir / 'bar.png'
    grouped = df.groupby('species')['body_mass_g'].agg(['mean','std']).reset_index()
    plt.figure(figsize=(8,6))
    bars = plt.bar(grouped['species'], grouped['mean'],
                   yerr=grouped['std'], capsize=6, color=plt.cm.Set2(np.linspace(0,1,len(grouped))))
    plt.title('Average Body Mass by Species (with Std Dev)')
    plt.ylabel('Body Mass (g)')
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x()+bar.get_width()/2, h, f'{h:.0f}', ha='center', va='bottom')
    plt.tight_layout()
    plt.savefig(fig3_path, dpi=150)
    plt.close()

    # Boxplot
    fig4_path = output_dir / 'boxplot.png'
    plt.figure(figsize=(10,4))
    sns.boxplot(x='species', y='bill_length_mm', data=df, palette='Set2')
    plt.title('Boxplot: Bill Length by Species')
    plt.tight_layout()
    plt.savefig(fig4_path, dpi=150)
    plt.close()

    # Heatmap
    fig5_path = output_dir / 'heatmap.png'
    plt.figure(figsize=(8,6))
    sns.heatmap(df[num_features].corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Heatmap (Numerical Features)')
    plt.tight_layout()
    plt.savefig(fig5_path, dpi=150)
    plt.close()

    return [fig1_path, fig2_path, fig3_path, fig4_path, fig5_path]

fig_paths = save_figures(OUT_DIR)

# -------------------------
# BUILD WORD DOCUMENT (python-docx)
# -------------------------
doc = Document()

# Styling basics
style = doc.styles['Normal']
style.font.name = 'Calibri'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Calibri')  # ensure fonts for all regions
style.font.size = Pt(11)

# Title
title = doc.add_heading('Penguin Interactive Dashboard — Project Report', level=0)
title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Introduction
doc.add_heading('1. Introduction', level=1)
doc.add_paragraph(
    "This report documents an interactive data visualization dashboard created using Matplotlib and ipywidgets in a Jupyter Notebook. "
    "The dashboard explores morphological features from the Palmer Penguins dataset (bill measurements, flipper length, body mass) across species and islands. "
    "The goal is to provide interactive exploration for both technical and non-technical audiences."
)

# Dataset
doc.add_heading('2. Dataset', level=1)
doc.add_paragraph(
    "Dataset: Palmer Penguins (from seaborn). Key columns used:\n"
    "- species: Adelie, Chinstrap, Gentoo\n"
    "- island: island location\n"
    "- bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g\n"
    "- sex"
)
doc.add_paragraph("Dataset size (after dropping missing rows): " + str(df.shape))

# Preprocessing
doc.add_heading('3. Data Cleaning & Preprocessing', level=1)
doc.add_paragraph(
    "Preprocessing steps performed:\n"
    "- Removed rows with missing values (df.dropna()) to avoid plotting/aggregation errors.\n"
    "- For production, consider imputation: median for numeric, mode for categorical."
)

# Methods / Dashboard Design
doc.add_heading('4. Methods & Dashboard Design', level=1)
doc.add_paragraph(
    "The dashboard contains three main interactive views:\n"
    "1. Scatter Plot: choose X/Y variables and color by species; filter by bill length.\n"
    "2. Histogram: visualize distribution of a selected numerical variable; adjustable bins and normalization.\n"
    "3. Bar Chart: group by species or island to view average metric with std deviation.\n"
    "Widgets: Dropdowns, FloatRangeSlider, IntSlider, Checkboxes, Reset button; arranged with HBox/VBox for a clean layout."
)

# Add figures with captions
doc.add_heading('5. Representative Figures', level=1)
doc.add_paragraph("Following images are static captures of the interactive dashboard's key plots.")
for p, cap in zip(fig_paths, [
    "Figure 1: Scatter plot — Flipper Length vs Body Mass by species",
    "Figure 2: Histogram — Body Mass distribution (mean shown)",
    "Figure 3: Bar chart — Average body mass by species (with std)",
    "Figure 4: Boxplot — Bill length by species",
    "Figure 5: Correlation heatmap of numerical features"
]):
    doc.add_picture(str(p), width=Inches(6.5))
    last_p = doc.paragraphs[-1]
    last_p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    c = doc.add_paragraph(cap)
    c.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    c.style = 'Intense Quote'

# Insights
doc.add_heading('6. Key Insights & Results', level=1)
doc.add_paragraph(
    "Key insights obtained by exploring the dashboard:\n"
    "- Gentoo penguins are generally larger and heavier than Adelie and Chinstrap.\n"
    "- Flipper length and body mass are positively correlated (larger flippers → higher mass).\n"
    "- Bill length and bill depth relationships differ by species and help separate species clusters.\n"
    "- Distribution shapes and variability differ across species; bar chart + std highlights which groups vary more."
)

# Conclusion
doc.add_heading('7. Conclusion', level=1)
doc.add_paragraph(
    "This dashboard demonstrates how Matplotlib and ipywidgets can produce a polished, interactive exploration environment. "
    "It is suitable for both data analysts and stakeholders to visually explore relationships and group differences."
)

# Suggestions & Future Work
doc.add_heading('8. Suggestions & Future Improvements', level=1)
doc.add_paragraph(
    "Suggested improvements:\n"
    "- Add hover tooltips (mplcursors) to show point-level details in scatter.\n"
    "- Add data export (download filtered CSV) to support analysis downstream.\n"
    "- Introduce true linked brushing using Bokeh/Plotly for richer interactivity.\n"
    "- Add classification model examples to show which features separate species best."
)

# Difficulties Faced
doc.add_heading('9. Difficulties Faced', level=1)
doc.add_paragraph(
    "Challenges encountered:\n"
    "- Managing duplicate plots in Jupyter (fixed via plt.close(fig) and explicit display).\n"
    "- ipywidgets compatibility across Notebook vs JupyterLab (nbextension vs prebuilt extensions).\n"
    "- Limitations in Matplotlib for advanced interactive behaviors compared to interactive JS-based libraries."
)

# Appendix & Reproducibility
doc.add_heading('10. Appendix: Code & Reproducibility', level=1)
doc.add_paragraph(
    "Run the Jupyter Notebook 'penguin_dashboard.ipynb' to reproduce the interactive dashboard. "
    "Required packages: pandas, numpy, seaborn, matplotlib, ipywidgets, python-docx. "
    "To regenerate the images and this report, re-run this script cell."
)

# Save the document
doc.save(DOC_PATH)
print("Report saved to:", DOC_PATH)
print("\nIf you are using Jupyter, download it from the file explorer or use:")
print(f" - Local path: {DOC_PATH.resolve()}")


C:\Users\HP\AppData\Local\Temp\ipykernel_5460\1325467725.py:90: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='species', y='bill_length_mm', data=df, palette='Set2')


Report saved to: penguin_report\Penguin_Dashboard_Report.docx

If you are using Jupyter, download it from the file explorer or use:
 - Local path: C:\Users\HP\Downloads\ALL Data science\Data Science Home Work\Internship _in_brainybeam\Third_task\Task_3\penguin_report\Penguin_Dashboard_Report.docx


In [94]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn

# -------------------------
# SETTINGS
# -------------------------
OUT_DIR = Path('./penguin_report')
OUT_DIR.mkdir(parents=True, exist_ok=True)
DOC_PATH = OUT_DIR / 'Penguin_Dashboard_Report_Custom.docx'

AUTHOR_NAME = "Mahek Bhayani"
PROJECT_DATE = datetime.today().strftime("%B %d, %Y")
LOGO_PATH = None  # <-- Optional: set to path of logo image file (e.g., "./logo.png")

# -------------------------
# LOAD DATA (with fallback)
# -------------------------
try:
    df = sns.load_dataset('penguins')
    if df is None:
        raise Exception("seaborn returned None")
except Exception:
    try:
        df = pd.read_csv('penguins.csv')
    except Exception:
        raise RuntimeError("Dataset could not be loaded. Provide 'penguins.csv' locally.")

df.dropna(inplace=True)
num_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# -------------------------
# GENERATE FIGURES
# -------------------------
def save_figures(output_dir):
    paths = {}
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='flipper_length_mm', y='body_mass_g', hue='species', s=80)
    plt.title('Scatter: Flipper Length vs Body Mass by Species')
    plt.tight_layout()
    paths['scatter'] = output_dir / 'scatter_custom.png'
    plt.savefig(paths['scatter'], dpi=150)
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.hist(df['body_mass_g'], bins=20, edgecolor='black', color='skyblue')
    plt.axvline(df['body_mass_g'].mean(), color='red', linestyle='dashed',
                label=f"Mean = {df['body_mass_g'].mean():.1f}")
    plt.title('Histogram: Body Mass (g)')
    plt.legend()
    plt.tight_layout()
    paths['histogram'] = output_dir / 'histogram_custom.png'
    plt.savefig(paths['histogram'], dpi=150)
    plt.close()

    plt.figure(figsize=(8, 6))
    grouped = df.groupby('species')['body_mass_g'].agg(['mean', 'std']).reset_index()
    bars = plt.bar(grouped['species'], grouped['mean'], yerr=grouped['std'],
                   capsize=6, color=plt.cm.Set2(np.linspace(0, 1, len(grouped))))
    plt.title('Average Body Mass by Species (with Std Dev)')
    plt.ylabel('Body Mass (g)')
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, h, f'{h:.0f}', ha='center', va='bottom')
    plt.tight_layout()
    paths['bar'] = output_dir / 'bar_custom.png'
    plt.savefig(paths['bar'], dpi=150)
    plt.close()

    plt.figure(figsize=(10, 4))
    sns.boxplot(x='species', y='bill_length_mm', data=df, palette='Set2')
    plt.title('Boxplot: Bill Length by Species')
    plt.tight_layout()
    paths['boxplot'] = output_dir / 'boxplot_custom.png'
    plt.savefig(paths['boxplot'], dpi=150)
    plt.close()

    plt.figure(figsize=(8, 6))
    sns.heatmap(df[num_features].corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Heatmap (Numerical Features)')
    plt.tight_layout()
    paths['heatmap'] = output_dir / 'heatmap_custom.png'
    plt.savefig(paths['heatmap'], dpi=150)
    plt.close()
    return paths

fig_paths = save_figures(OUT_DIR)

# -------------------------
# BUILD WORD DOCUMENT
# -------------------------
doc = Document()
style = doc.styles['Normal']
style.font.name = 'Calibri'
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Calibri')
style.font.size = Pt(11)

# Logo (if provided)
if LOGO_PATH and Path(LOGO_PATH).exists():
    doc.add_picture(LOGO_PATH, width=Inches(1.5))
    doc.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Title Page
title = doc.add_heading('Penguin Interactive Dashboard — Project Report', level=0)
title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
doc.add_paragraph(f"Author: {AUTHOR_NAME}\nDate: {PROJECT_DATE}", style='Intense Quote').alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

doc.add_page_break()

# Introduction
doc.add_heading('1. Introduction', level=1)
doc.add_paragraph(
    f"This report presents the development of an interactive dashboard for exploring the Palmer Penguins dataset. "
    f"Created by {AUTHOR_NAME}, the goal of this project is to allow users to visually analyze penguin morphology data, "
    "discover patterns across species and islands, and draw actionable insights."
)

# Dataset
doc.add_heading('2. Dataset', level=1)
doc.add_paragraph(
    "The Palmer Penguins dataset is a modern alternative to the classic Iris dataset, containing real-world measurements:\n"
    "- Species: Adelie, Chinstrap, Gentoo\n"
    "- Island: Dream, Biscoe, Torgersen\n"
    "- Bill length & depth (mm), Flipper length (mm), Body mass (g), Sex"
)
doc.add_paragraph(f"Dataset shape after cleaning: {df.shape[0]} rows × {df.shape[1]} columns")

# Figures with captions
doc.add_heading('3. Key Visualizations', level=1)
for key, caption in zip(fig_paths.keys(), [
    "Scatter Plot: Flipper Length vs Body Mass by Species",
    "Histogram: Body Mass Distribution with Mean",
    "Bar Chart: Average Body Mass by Species",
    "Boxplot: Bill Length by Species",
    "Correlation Heatmap of Numerical Features"
]):
    doc.add_picture(str(fig_paths[key]), width=Inches(6.5))
    p = doc.add_paragraph(caption, style='Intense Quote')
    p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Insights
doc.add_heading('4. Key Insights', level=1)
doc.add_paragraph(
    "- Gentoo penguins are significantly larger and heavier than Adelie and Chinstrap.\n"
    "- Strong positive correlation between flipper length and body mass.\n"
    "- Bill length and depth separate species well, aiding classification.\n"
    "- Island-level grouping shows habitat influence on penguin morphology."
)

# Challenges & Suggestions
doc.add_heading('5. Challenges & Suggestions', level=1)
doc.add_paragraph(
    "Challenges faced:\n"
    "- Handling missing values (resolved by dropping rows).\n"
    "- Ensuring widgets display correctly in Jupyter Notebook (resolved via nbextension enable).\n"
    "- Avoiding duplicate plots (fixed using plt.close(fig)).\n\n"
    "Suggestions for improvement:\n"
    "- Add interactive hover tooltips.\n"
    "- Allow exporting filtered data to CSV.\n"
    "- Include more advanced statistical analysis (ANOVA, regression)."
)

# Conclusion
doc.add_heading('6. Conclusion', level=1)
doc.add_paragraph(
    f"This project successfully demonstrates the power of interactive data visualization with Matplotlib and ipywidgets. "
    f"Thanks to the clear interface, users can gain quick insights into penguin characteristics, species differences, "
    "and feature correlations. This approach can be adapted to other datasets for teaching, reporting, and exploratory analysis."
)

doc.save(DOC_PATH)
DOC_PATH


C:\Users\HP\AppData\Local\Temp\ipykernel_5460\643063869.py:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='species', y='bill_length_mm', data=df, palette='Set2')


WindowsPath('penguin_report/Penguin_Dashboard_Report_Custom.docx')